# Language Models, the Chat Format and Tokens

## Setup

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir("drive/")
os.chdir('My Drive')
os.chdir('Experiment')
os.chdir('ChatGPT')

In [ ]:
OUTPUT_DIR = './outputs/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.1 MB/s eta 0:00:00


In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

#### helper function

Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat). 

This helper function will make it easier to use prompts and look at the generated outputs:

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message["content"]

In [ ]:
def get_completion_and_token_count(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message["content"]
    
    token_dict = {
      'prompt_tokens':response['usage']['prompt_tokens'],
      'completion_tokens':response['usage']['completion_tokens'],
      'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

## Prompt the model and get a completion

In [ ]:
response = get_completion("What is the capital of France?")
print(response)

The capital of France is Paris.


## Tokens

In [ ]:
response = get_completion("Take the letters in lollipop and reverse them")
print(response)

ppilolol


"lollipop" in reverse should be "popillol"

In [ ]:
response = get_completion("""Take the letters in l-o-l-l-i-p-o-p and reverse them""")
print(response)

p-o-p-i-l-l-o-l


## With formatted messages

In [ ]:
messages =  [  
  {'role':'system', 
  'content':"""You are an assistant who responds in the style of Dr Seuss."""},    
  {'role':'user', 
  'content':"""write me a very short poem about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

Oh, the happy orange carrot,
With a smile as big as a parrot,
Growing in the garden so green,
Oh, how happy it has been!


In [ ]:
# length
messages =  [  
  {'role':'system',
  'content':'All your responses must be \
  one sentence long.'},    
  {'role':'user',
  'content':'write me a story about a happy carrot'},  
]
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once upon a time, there lived a happy carrot named Carl who lived in a beautiful garden and loved nothing more than basking in the warm sun and soaking up the fresh rainwater.


In [ ]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
]
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once there was a carrot named Carl who grew to be plump and bright, always gleeful in his patch and when he was harvested, he became the starring ingredient in a delicious bowl of soup that brought smiles to everyone who tasted it.


In [ ]:
# combined
messages =  [  
{'role':'system',
 'content':"""You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

In a garden so bright, lived a carrot so cheery and light; he loved chatting with the bees and the birds, and every morning he smiled at the rising sun without any words.


In [ ]:
messages = [
  {'role':'system', 
  'content':"""You are an assistant who responds\
  in the style of Dr Seuss."""},    
  {'role':'user',
  'content':"""write me a very short poem \ 
  about a happy carrot"""},  
]


response, token_dict = get_completion_and_token_count(messages)

In [ ]:
print(response)

Oh, the happy carrot, so bright and so bold,
With a smile on its face, and a story untold.
It grew in the garden, with love and with care,
And now it's so happy, it's beyond compare!


In [ ]:
print(token_dict)

{'prompt_tokens': 41, 'completion_tokens': 49, 'total_tokens': 90}


#### A note about the backslash

- In the notebook, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.